In [5]:
from google.colab import files
uploaded = files.upload()


Saving Score.csv to Score (1).csv
Saving Iris.csv to Iris (1).csv


In [6]:
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
from math import log, exp
import random
from itertools import combinations
from tqdm import tqdm_notebook as tqdm

In [7]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

def train_val_test_split(X, y, train_frac=0.8, val_frac=0.1, seed=RANDOM_SEED):
    assert len(X) == len(y)
    n = len(X)
    idx = np.arange(n)
    np.random.seed(seed)
    np.random.shuffle(idx)
    Xs = X[idx]
    ys = y[idx]
    n_train = int(n * train_frac)
    n_val = int(n * val_frac)
    X_train = Xs[:n_train]
    y_train = ys[:n_train]
    X_val = Xs[n_train:n_train+n_val]
    y_val = ys[n_train:n_train+n_val]
    X_test = Xs[n_train+n_val:]
    y_test = ys[n_train+n_val:]
    return X_train, X_val, X_test, y_train, y_val, y_test

    # ========== SIMPLE PREPROCESS =============
def encode_labels(y):
    classes, inv = np.unique(y, return_inverse=True)
    return inv, {i:cls for i,cls in enumerate(classes)}


In [8]:
# ========= Q1/Q2: DECISION STUMP (from scratch) ==========
class DecisionStump:
    """
    Axis-aligned decision stump for multiclass classification.
    Learns a threshold on one feature and assigns a class label to each side.
    Supports weighted training (sample weights).
    """
    def __init__(self):
        self.feature_index = None
        self.threshold = None
        self.left_class = None
        self.right_class = None
        self.polarity = 1  # unused but kept
        self.default_class = None

    def fit(self, X, y, sample_weights=None, n_thresholds=20):
        # X: (n_samples, n_features), y: integer class labels 0..K-1
        n, d = X.shape
        if sample_weights is None:
            sample_weights = np.ones(n) / n
        best_err = float('inf')
        self.default_class = Counter(y).most_common(1)[0][0]
        # For each feature, try thresholds
        for j in range(d):
            vals = X[:, j]
            # consider thresholds = quantiles to speed up
            thresholds = np.unique(np.percentile(vals, np.linspace(0, 100, n_thresholds)))
            for t in thresholds:
                # left = vals <= t, right = vals > t
                left_idx = vals <= t
                right_idx = ~left_idx
                if left_idx.sum() == 0 or right_idx.sum() == 0:
                    continue
                # find weighted majority class in left and right
                def weighted_majority(idx):
                    if idx.sum() == 0:
                        return self.default_class
                    w = defaultdict(float)
                    for k,ww in zip(y[idx], sample_weights[idx]):
                        w[int(k)] += float(ww)
                    return max(w.items(), key=lambda x: x[1])[0]
                left_class = weighted_majority(left_idx)
                right_class = weighted_majority(right_idx)
                # compute weighted error
                preds = np.where(left_idx, left_class, right_class)
                err = np.sum(sample_weights * (preds != y))
                if err < best_err:
                    best_err = err
                    self.feature_index = j
                    self.threshold = t
                    self.left_class = left_class
                    self.right_class = right_class
        # If we didn't find any split, set to majority class
        if self.feature_index is None:
            self.feature_index = 0
            self.threshold = X[:,0].mean()
            self.left_class = self.default_class
            self.right_class = self.default_class
        return self

    def predict(self, X):
        vals = X[:, self.feature_index]
        return np.where(vals <= self.threshold, self.left_class, self.right_class)

    def predict_proba(self, X, y_train=None, sample_weights=None):
        # We'll approximate class probability by returning one-hot of predicted class
        preds = self.predict(X)
        K = int(max(preds.max(), 0) + 1) if y_train is None else int(np.max(y_train)+1)
        proba = np.zeros((len(X), K))
        for i,c in enumerate(preds):
            proba[i, int(c)] = 1.0
        return proba

In [9]:
# ======== AdaBoost - SAMME (discrete multiclass) - from scratch =========
class AdaBoostSAMME:
    def __init__(self, n_estimators=50):
        self.n_estimators = n_estimators
        self.learners = []
        self.alphas = []
        self.classes_ = None

    def fit(self, X, y):
        n_samples = X.shape[0]
        self.classes_ = np.unique(y)
        K = len(self.classes_)
        # Initialize weights
        w = np.ones(n_samples) / n_samples
        for m in range(self.n_estimators):
            stump = DecisionStump()
            stump.fit(X, y, sample_weights=w, n_thresholds=25)
            pred = stump.predict(X)
            # weighted error
            incorrect = (pred != y).astype(float)
            err = np.sum(w * incorrect) / np.sum(w)
            # avoid divide by zero / numeric problems
            err = max(1e-10, min(1-1e-10, err))
            # alpha for SAMME (discrete multiclass)
            alpha = np.log((1 - err) / err) + np.log(K - 1)
            # update weights
            w = w * np.exp(alpha * incorrect)
            w = w / np.sum(w)
            self.learners.append(stump)
            self.alphas.append(alpha)
        self.alphas = np.array(self.alphas)
        return self

    def predict(self, X):
        # compute class scores
        K = len(self.classes_)
        scores = np.zeros((X.shape[0], K))
        for alpha, learner in zip(self.alphas, self.learners):
            preds = learner.predict(X).astype(int)
            for i,p in enumerate(preds):
                scores[i, p] += alpha
        return np.argmax(scores, axis=1)

In [10]:
# ======= AdaBoost SAMME.R-like (Real AdaBoost multiclass) ==========
class AdaBoostSAMMER:
    """
    Implements a SAMME.R-like real-valued AdaBoost for multiclass.
    We use the stump's 'probability' estimate from training set leaves to produce log odds.
    """
    def __init__(self, n_estimators=50):
        self.n_estimators = n_estimators
        self.learners = []
        self.estimator_class_probas = []  # for each learner, mapping from leaf to class prob
        self.classes_ = None
        self.alphas = []

    def fit(self, X, y):
        n = X.shape[0]
        self.classes_ = np.unique(y)
        K = len(self.classes_)
        w = np.ones(n) / n
        for m in range(self.n_estimators):
            stump = DecisionStump()
            # For real boosting, when training stump we need class probabilities on each leaf weighted by w.
            stump.fit(X, y, sample_weights=w, n_thresholds=25)
            # compute per-sample class probability estimates using weighted majority on each leaf
            # We'll build mapping: for values <= thresh -> distribution over classes; > thresh -> distribution
            feat = stump.feature_index
            th = stump.threshold
            left_idx = X[:,feat] <= th
            right_idx = ~left_idx
            # helper to compute weighted class probs
            def class_prob(idx):
                prob = defaultdict(float)
                s = np.sum(w[idx]) if np.sum(w[idx])>0 else 1.0
                for lab, wt in zip(y[idx], w[idx]):
                    prob[int(lab)] += wt
                for k in list(prob.keys()):
                    prob[k] /= s
                return prob
            left_prob = class_prob(left_idx)
            right_prob = class_prob(right_idx)
            # store these distributions
            self.estimator_class_probas.append((left_prob, right_prob, feat, th))
            # For prediction on training set produce probs per sample
            probs = np.zeros((n, K))
            for i in range(n):
                if left_idx[i]:
                    for k,v in left_prob.items():
                        probs[i,k] = v
                else:
                    for k,v in right_prob.items():
                        probs[i,k] = v
            # avoid zeros
            probs = np.clip(probs, 1e-12, 1.0)
            # compute pseudo-response: (K-1)/K * (log p_k - average log p)
            # Following Friedman et al. derivation for SAMME.R, alpha_m = 1
            # We'll compute contribution to class scores as 0.5 * log(p_k)
            # Simpler stable approach: compute estimator's class score for each sample: s_k = log(p_k)
            scores = np.log(probs)
            # compute the loss/error to update weights
            # exponent factor for sample i :
            # w_i <- w_i * exp(- (1/K) * sum_k y_{i,k} * s_{i,k}) but we use simpler: increase weight when true class has low p
            true_probs = probs[np.arange(n), y]
            err = 1 - np.sum(w * true_probs) / np.sum(w)
            err = max(1e-12, min(1-1e-12, err))
            # set alpha proportional to 0.5*log((1-err)/err) as proxy
            alpha = 0.5 * np.log((1-err)/err)
            # update weights using class probability on true class (raise weight where prob low)
            w = w * np.exp(-alpha * (scores[np.arange(n), y] - np.log(1.0/K)))
            w = w / np.sum(w)
            self.learners.append(stump)
            self.alphas.append(alpha)
        self.alphas = np.array(self.alphas)
        return self

    def predict(self, X):
        K = len(self.classes_)
        n = X.shape[0]
        scores = np.zeros((n, K))
        for alpha, (left_prob, right_prob, feat, th) in zip(self.alphas, self.estimator_class_probas):
            # construct per-sample scores = alpha * log p_k
            probs = np.zeros((n, K))
            left_idx = X[:,feat] <= th
            for i in range(n):
                if left_idx[i]:
                    for k,v in left_prob.items():
                        probs[i,k] = v
                else:
                    for k,v in right_prob.items():
                        probs[i,k] = v
            probs = np.clip(probs, 1e-12, 1.0)
            scores += alpha * np.log(probs)
        return np.argmax(scores, axis=1)

In [11]:
# ======= EVALUATION METRICS: accuracy & classification report ========
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def classification_report_simple(y_true, y_pred, label_map=None):
    labels = np.unique(np.concatenate([y_true, y_pred]))
    out = {}
    for lab in labels:
        tp = np.sum((y_pred==lab)&(y_true==lab))
        fp = np.sum((y_pred==lab)&(y_true!=lab))
        fn = np.sum((y_pred!=lab)&(y_true==lab))
        prec = tp/(tp+fp) if tp+fp>0 else 0.0
        rec = tp/(tp+fn) if tp+fn>0 else 0.0
        f1 = 2*prec*rec/(prec+rec) if prec+rec>0 else 0.0
        out[lab] = {"precision":prec, "recall":rec, "f1":f1, "support":np.sum(y_true==lab)}
    return out

In [12]:
# ========= Q3: DBSCAN from scratch ============
def euclidean(a,b):
    return np.sqrt(((a-b)**2).sum())

def dbscan(X, eps=0.5, min_samples=5):
    n = len(X)
    labels = -1 * np.ones(n, dtype=int)
    visited = np.zeros(n, dtype=bool)
    cluster_id = 0
    # precompute neighbors via vectorized approach for speed (distance matrix)
    # but careful with memory; for moderate datasets it's OK (Iris small). For bigger use KDTree.
    D = np.sqrt(((X[:,None,:] - X[None,:,:])**2).sum(axis=2))
    for i in range(n):
        if visited[i]:
            continue
        visited[i] = True
        neighbors = np.where(D[i] <= eps)[0]
        if len(neighbors) < min_samples:
            labels[i] = -1  # noise (may be changed later)
        else:
            # create new cluster
            labels[i] = cluster_id
            seeds = list(neighbors[neighbors != i])
            while seeds:
                s = seeds.pop(0)
                if not visited[s]:
                    visited[s] = True
                    neigh_s = np.where(D[s] <= eps)[0]
                    if len(neigh_s) >= min_samples:
                        # add new neighbors to seeds if not already in seeds
                        for nb in neigh_s:
                            if nb not in seeds:
                                seeds.append(nb)
                if labels[s] == -1:
                    labels[s] = cluster_id
                elif labels[s] is None:
                    labels[s] = cluster_id
            cluster_id += 1
    # some labels may be -1 already; others are >=0
    return labels

# ======= Silhouette Score from scratch ==========
def silhouette_score(X, labels):
    # labels: -1 for noise; compute silhouette for non-noise only
    n = len(X)
    unique_labels = [l for l in np.unique(labels) if l!=-1]
    if len(unique_labels) < 2:
        return None
    D = np.sqrt(((X[:,None,:] - X[None,:,:])**2).sum(axis=2))
    sil_samples = []
    for i in range(n):
        if labels[i] == -1:
            continue
        same = np.where(labels == labels[i])[0]
        other_labels = [l for l in unique_labels if l != labels[i]]
        if len(same) == 1:
            a = 0.0
        else:
            a = np.mean(D[i, same[same != i]])
        b_vals = []
        for ol in other_labels:
            ol_idx = np.where(labels == ol)[0]
            if len(ol_idx) > 0:
                b_vals.append(np.mean(D[i, ol_idx]))
        b = min(b_vals) if b_vals else 0.0
        denom = max(a,b)
        if denom == 0:
            sil = 0.0
        else:
            sil = (b - a) / denom
        sil_samples.append(sil)
    if len(sil_samples) == 0:
        return None
    return float(np.mean(sil_samples))

# ======= Adjusted Rand Index (from scratch) ==========
def adjusted_rand_index(labels_true, labels_pred):
    # based on combinatorial formula
    n = len(labels_true)
    # build contingency table
    label_true_ids = {}
    label_pred_ids = {}
    for i,l in enumerate(labels_true):
        label_true_ids.setdefault(l, []).append(i)
    for i,l in enumerate(labels_pred):
        label_pred_ids.setdefault(l, []).append(i)
    # contingency matrix counts
    import math
    sum_comb_c = 0.0
    sum_comb_k = 0.0
    for key, idxs in label_true_ids.items():
        c = len(idxs)
        sum_comb_c += math.comb(c,2) if c>=2 else 0
    for key, idxs in label_pred_ids.items():
        k = len(idxs)
        sum_comb_k += math.comb(k,2) if k>=2 else 0
    # sum of combinations for intersections
    sum_comb = 0.0
    for ct, idxs_t in label_true_ids.items():
        set_t = set(idxs_t)
        for cp, idxs_p in label_pred_ids.items():
            inter = len(set_t.intersection(idxs_p))
            if inter>=2:
                sum_comb += math.comb(inter,2)
    total_comb = math.comb(n,2)
    expected_index = (sum_comb_c * sum_comb_k) / total_comb if total_comb>0 else 0
    max_index = 0.5 * (sum_comb_c + sum_comb_k)
    ari = (sum_comb - expected_index) / (max_index - expected_index) if (max_index - expected_index) != 0 else 0.0
    return ari

# ========== Q4: Perceptron / Neural nets from scratch ===========
def one_hot(y, K=None):
    if K is None:
        K = len(np.unique(y))
    arr = np.zeros((len(y), K))
    for i,lab in enumerate(y):
        arr[i, lab] = 1
    return arr

def softmax(z):
    z = z - np.max(z, axis=1, keepdims=True)
    ez = np.exp(z)
    return ez / np.sum(ez, axis=1, keepdims=True)

def cross_entropy_loss(probs, y_true_onehot):
    # probs: (n,K)
    n = probs.shape[0]
    probs = np.clip(probs, 1e-12, 1-1e-12)
    return -np.sum(y_true_onehot * np.log(probs)) / n

class MLP:
    def __init__(self, layer_sizes, lr=0.01, epochs=200, batch_size=16, seed=RANDOM_SEED):
        # layer_sizes: e.g. [D, H, K] where D input dim, K output dim
        self.layer_sizes = layer_sizes
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        self.params = {}
        np.random.seed(seed)
        # initialize weights
        for i in range(len(layer_sizes)-1):
            in_dim = layer_sizes[i]
            out_dim = layer_sizes[i+1]
            # Xavier init
            limit = np.sqrt(6.0/(in_dim+out_dim))
            self.params['W'+str(i+1)] = np.random.uniform(-limit, limit, (in_dim, out_dim))
            self.params['b'+str(i+1)] = np.zeros((1, out_dim))

    def _forward(self, X):
        caches = {}
        A = X
        L = len(self.layer_sizes)-1
        for l in range(1, L+1):
            W = self.params['W'+str(l)]
            b = self.params['b'+str(l)]
            Z = A.dot(W) + b  # (n, out_dim)
            if l < L:
                A = np.tanh(Z)   # hidden activation
            else:
                A = softmax(Z)   # last layer softmax
            caches['A'+str(l-1)] = (None if l==1 else caches['A'+str(l-2)][0])  # not used
            caches['Z'+str(l)] = Z
            caches['A'+str(l)] = A
        return A, caches

    def _backward(self, X, y_onehot):
        grads = {}
        n = X.shape[0]
        L = len(self.layer_sizes)-1
        # forward to get activations step by step since we didn't save them nicely
        A = [None]*(L+1)
        Z = [None]*(L+1)
        A[0] = X
        for l in range(1, L+1):
            W = self.params['W'+str(l)]
            b = self.params['b'+str(l)]
            Z[l] = A[l-1].dot(W) + b
            if l < L:
                A[l] = np.tanh(Z[l])
            else:
                A[l] = softmax(Z[l])
        # gradient on output
        dA = (A[L] - y_onehot) / n  # (n,K)
        for l in reversed(range(1, L+1)):
            dZ = dA
            grads['W'+str(l)] = A[l-1].T.dot(dZ)
            grads['b'+str(l)] = np.sum(dZ, axis=0, keepdims=True)
            if l-1 > 0:
                dA = dZ.dot(self.params['W'+str(l)].T) * (1 - np.tanh(Z[l-1])**2)
            else:
                dA = dZ.dot(self.params['W'+str(l)].T)
        return grads

    def fit(self, X, y, X_val=None, y_val=None, verbose=False):
        n = X.shape[0]
        K = self.layer_sizes[-1]
        y_onehot = one_hot(y, K)
        for epoch in range(self.epochs):
            # shuffle
            idx = np.arange(n)
            np.random.shuffle(idx)
            for start in range(0, n, self.batch_size):
                batch_idx = idx[start:start+self.batch_size]
                xb = X[batch_idx]
                yb = y_onehot[batch_idx]
                # forward
                probs, _ = self._forward(xb)
                # backward
                grads = self._backward(xb, yb)
                # update
                for k in grads:
                    self.params[k] -= self.lr * grads[k]
            if verbose and (epoch%50==0 or epoch==self.epochs-1):
                train_pred = self.predict(X)
                train_acc = np.mean(train_pred == y)
                if X_val is not None:
                    val_pred = self.predict(X_val)
                    val_acc = np.mean(val_pred == y_val)
                    print(f"Epoch {epoch}: train_acc={train_acc:.4f}, val_acc={val_acc:.4f}")
                else:
                    print(f"Epoch {epoch}: train_acc={train_acc:.4f}")
        return self

    def predict_proba(self, X):
        probs, _ = self._forward(X)
        return probs

    def predict(self, X):
        probs = self.predict_proba(X)
        return np.argmax(probs, axis=1)

In [ ]:
# ========== RUN: Q1 & Q2 on Credit Score CSV ==========
# Load Score.csv from /content (you uploaded file to Colab)
score_df = pd.read_csv('/content/Score.csv')  # change path if needed
# Simple preprocessing: label encode categorical columns
for col in score_df.columns:
    if score_df[col].dtype == 'object':
        score_df[col] = pd.factorize(score_df[col])[0]
# assume the label column is named 'Credit_Score' or last column
if 'Credit_Score' in score_df.columns:
    label_col = 'Credit_Score'
else:
    label_col = score_df.columns[-1]
X = score_df.drop(label_col, axis=1).values.astype(float)
y_raw = score_df[label_col].values
y, inv_map = encode_labels(y_raw)

# split
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X, y, train_frac=0.8, val_frac=0.1, seed=RANDOM_SEED)
print("Score dataset shapes:", X_train.shape, X_val.shape, X_test.shape)

# Train SAMME (discrete)
samme = AdaBoostSAMME(n_estimators=100)
samme.fit(X_train, y_train)
y_pred_test_samme = samme.predict(X_test)
print("SAMME Test accuracy:", accuracy(y_test, y_pred_test_samme))
print("SAMME classification report:", classification_report_simple(y_test, y_pred_test_samme))

# Train SAMME.R-like (real)
sammer = AdaBoostSAMMER(n_estimators=100)
sammer.fit(X_train, y_train)
y_pred_test_sammer = sammer.predict(X_test)
print("SAMME.R-like Test accuracy:", accuracy(y_test, y_pred_test_sammer))
print("SAMME.R-like classification report:", classification_report_simple(y_test, y_pred_test_sammer))


Score dataset shapes: (79968, 20) (9996, 20) (9996, 20)


In [ ]:
# ========== RUN: Q3 DBSCAN & silhouette on Iris ==========
iris_df = pd.read_csv('/content/Iris.csv')  # change path if needed
# Some Iris CSV from Kaggle has a column 'Id' or unnamed first; drop non-feature cols
possible_label_cols = ['Species','species','class','label']
label_col_iris = None
for c in iris_df.columns:
    if c in possible_label_cols:
        label_col_iris = c
        break
if label_col_iris is None:
    # assume last column is label
    label_col_iris = iris_df.columns[-1]
# features are numeric columns
feature_cols = [c for c in iris_df.columns if c != label_col_iris]
X_iris = iris_df[feature_cols].values.astype(float)
y_iris_raw = iris_df[label_col_iris].values
y_iris, iris_map = encode_labels(y_iris_raw)

# Standardize features
X_iris = (X_iris - X_iris.mean(axis=0)) / (X_iris.std(axis=0) + 1e-12)

# Run DBSCAN
eps = 0.5
min_samples = 5
labels_db = dbscan(X_iris, eps=eps, min_samples=min_samples)
print("DBSCAN labels found:", np.unique(labels_db))
ari = adjusted_rand_index(y_iris, labels_db)
sil = silhouette_score(X_iris, labels_db)
print("DBSCAN ARI:", ari)
print("DBSCAN Silhouette:", sil)


In [ ]:
# ========== RUN: Q4 Perceptrons (0,1,2 hidden layers) on Iris ==========
# split iris into train/test
Xtr, Xv, Xt, ytr, yv, yt = train_val_test_split(X_iris, y_iris, train_frac=0.8, val_frac=0.0, seed=RANDOM_SEED)
# note: we used val_frac=0.0 to get 80:0:20; modify if you want val set
D = Xtr.shape[1]
K = len(np.unique(y_iris))

# 0 hidden -> logistic softmax (MLP with no hidden layers means W shape (D,K))
model0 = MLP([D, K], lr=0.05, epochs=400, batch_size=16)
model0.fit(Xtr, ytr, X_val=Xt, y_val=yt, verbose=False)
acc0 = accuracy(yt, model0.predict(Xt))

# 1 hidden
model1 = MLP([D, 8, K], lr=0.05, epochs=400, batch_size=16)
model1.fit(Xtr, ytr, X_val=Xt, y_val=yt, verbose=False)
acc1 = accuracy(yt, model1.predict(Xt))

# 2 hidden
model2 = MLP([D, 8, 8, K], lr=0.05, epochs=400, batch_size=16)
model2.fit(Xtr, ytr, X_val=Xt, y_val=yt, verbose=False)
acc2 = accuracy(yt, model2.predict(Xt))

print("Perceptron 0 hidden acc:", acc0)
print("Perceptron 1 hidden acc:", acc1)
print("Perceptron 2 hidden acc:", acc2)

In [ ]:
print("\n=== SUMMARY ===")
print("CreditScore SAMME test acc:", accuracy(y_test, y_pred_test_samme))
print("CreditScore SAMME.R-like test acc:", accuracy(y_test, y_pred_test_sammer))
print("Iris DBSCAN ARI:", ari, "Silhouette:", sil)
print("Iris perceptron accuracies (0,1,2 hidden):", acc0, acc1, acc2)